In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pytz
from nltk.probability import FreqDist
from custom_scripts import *
%matplotlib inline

# Trading on Sentiment

### Data Cleaning Notebook

In this notebook we will address the data cleaning steps needed in order to have a dataset suitable for modeling and analysis. We will import all of the yearly articles, convert the datetimes to Eastern Standard Time, tag the sentiment of each article, and combine each article with its historical stock price information for the day it was published. Finally we will tokenize, remove stop words, then aggregate all of the values so that we have one row of information per day. 

##### Import yearly data.

In [2]:
df1 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2020.csv', index_col=0)
df2 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2019.csv', index_col=0)
df3 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2018.csv', index_col=0)
df4 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2017.csv', index_col=0)
df5 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2016.csv', index_col=0)
df6 = pd.read_csv('/Volumes/pimyllifeupshare/training_data/Goldman Sachs2015.csv', index_col=0)
df = pd.concat([df1,df2, df3, df4, df5, df6])
df.dropna(subset=['fulltext'], inplace=True)
df.shape

(37709, 7)

##### Clean newlines and special characters

In [3]:
%%time
df['cleaned_text'] = df['fulltext'].apply(clean_text)
df['cleaned_authors'] = df['author'].apply(clean_text)

CPU times: user 28 s, sys: 595 ms, total: 28.6 s
Wall time: 28.8 s


##### Changing the UTC time to EST.

In [4]:
#Change to Datetime
df["date"]= pd.to_datetime(df["date"])
df = df.set_index('date')
df.index = df.index.normalize()

In [5]:
%%time
#convert DateTime index to eastern time. 
eastern = pytz.timezone('US/Eastern')
df.index = df.index.tz_convert(eastern).tz_localize(None)
#put into year/month/day format
df.index = df.index.strftime('%Y-%m-%d')

CPU times: user 262 ms, sys: 5.26 ms, total: 268 ms
Wall time: 275 ms


### Using [tldextract](https://pypi.org/project/tldextract/) to extract company names from url's

In [6]:
%%time
df['news_outlet'] = df['source'].apply(get_outlet)
print('The dataset contains {} different articles from {} news outlets \n'.format(df.shape[0],df.news_outlet.nunique()))

The dataset contains 37709 different articles from 595 news outlets 

CPU times: user 365 ms, sys: 7.48 ms, total: 373 ms
Wall time: 377 ms


# Getting historical Stock Prices

In [7]:
years_we_need = [2015, 2016, 2017, 2018, 2019, 2020]

full_date_list = []

for year in years_we_need:
    res = get_month_day_range(year)
    full_date_list += res

The `get_past_prices` custom function uses a list of dates and a ticker symbol to call the twelvedata.com API for all of the dates in the provided list. In our case we want historical prices over the past 5 years, because we have 5 years worth of articles.

In [8]:
historical_prices = get_past_prices(full_date_list, 'GS')

1) 2015-01-01 to 2015-01-31
2) 2015-02-01 to 2015-02-28
3) 2015-03-01 to 2015-03-31
4) 2015-04-01 to 2015-04-30
5) 2015-05-01 to 2015-05-31
6) 2015-06-01 to 2015-06-30
7) 2015-07-01 to 2015-07-31
8) 2015-08-01 to 2015-08-31
9) 2015-09-01 to 2015-09-30
10) 2015-10-01 to 2015-10-31
11) 2015-11-01 to 2015-11-30
12) 2015-12-01 to 2015-12-31
13) 2016-01-01 to 2016-01-31
14) 2016-02-01 to 2016-02-29
15) 2016-03-01 to 2016-03-31
16) 2016-04-01 to 2016-04-30
17) 2016-05-01 to 2016-05-31
18) 2016-06-01 to 2016-06-30
19) 2016-07-01 to 2016-07-31
20) 2016-08-01 to 2016-08-31
21) 2016-09-01 to 2016-09-30
22) 2016-10-01 to 2016-10-31
23) 2016-11-01 to 2016-11-30
24) 2016-12-01 to 2016-12-31
25) 2017-01-01 to 2017-01-31
26) 2017-02-01 to 2017-02-28
27) 2017-03-01 to 2017-03-31
28) 2017-04-01 to 2017-04-30
29) 2017-05-01 to 2017-05-31
30) 2017-06-01 to 2017-06-30
31) 2017-07-01 to 2017-07-31
32) 2017-08-01 to 2017-08-31
33) 2017-09-01 to 2017-09-30
34) 2017-10-01 to 2017-10-31
35) 2017-11-01 to 2017-

The loop below iterates through the historical prices and calculates the change in a stock price from one open to another. Adding a 0 if the stock decreased or there was not change, and adding a 1 if the stock increased. This is an initial tagging step, the threshold for targets can be adjusted later using the 'day_change' column.

In [9]:
prices_index = historical_prices.index.strftime('%Y-%m-%d').to_list()

In [10]:
historical_prices.sort_index(inplace=True)

In [11]:
df_res = pd.DataFrame(columns = ['day_change', 'increase', 'date'])
for i,stock_price in enumerate(prices_index):
    try:
        today = historical_prices.loc[prices_index[i]].open
        tomorrow = historical_prices.loc[prices_index[i+1]].open
        direction = tomorrow - today
        if direction < 0:
            increase = 0
        else:
            increase = 1
        df_res = df_res.append({'day_change': direction, 'increase':increase, 'date':stock_price}, ignore_index=True)
    except Exception as e:
        continue

In [12]:
df_res.tail(5)

,day_change,increase,date
1500,1.50999,1,2020-12-08
1501,-1.09999,0,2020-12-07
1502,-0.15001,0,2020-12-04
1503,-5.20999,0,2020-12-03
1504,-0.11999,0,2020-12-02


In [13]:
df_res["date"]= pd.to_datetime(df_res["date"])
df_res = df_res.set_index('date')

In [14]:
df_res.sort_index(inplace=True)
targets = df_res

In [15]:
#merge the historical prices with the daily change we calculated and the targets. 
targs=pd.merge(targets,historical_prices, how='outer', left_index=True, right_index=True)

In [16]:
targs.tail(4)

,day_change,increase,open,high,low,close,volume
2020-12-24,-5.04999,0,256.54999,257.57999,253.75000,256.16000,957649
2020-12-28,-1.26001,0,257.81000,262.65500,257.00000,259.59000,2732245
2020-12-29,-2.45001,0,260.26001,260.85999,256.50000,258.01001,1430400
2020-12-30,1.45001,1,258.81000,260.64999,257.82999,259.45001,1566500


In [17]:
#Save the targets
targs.to_csv('gs_targs.csv')

In [18]:
#Merge targets and main data on the date
df=pd.merge(df,targs, how='outer', left_index=True, right_index=True)

To account for weekends and holidays when the market is closed. Forward filling of the previous non-NA value is used. 

In [19]:
df.fillna(method='ffill', inplace = True)

#Drop the few late 2014 values where we have not price data. 
df.dropna(subset=['increase', 'open', 'high', 'low', 'close'], inplace = True)

# Predict Sentiment for each Article with VADER

To tag the sentiment of each article, we will use the [VADER](https://github.com/cjhutto/vaderSentiment) sentiment analyzer. The `sentiment_analyzer_scores` custom function inputs a string and output the result of the VADER sentiment prediction. Vader is primarily used for social media text; however, is effective with news articles as well. 

In [20]:
%%time

#tag the sentiment for each article using VADER. This will take a few minutes.
df['sentiment'] = df['fulltext'].apply(sentiment_analyzer_scores)

KeyboardInterrupt: 

After predicting sentiment of the article, we can create dummies of the values.

In [ ]:
sentiment_dummies = pd.get_dummies(df['sentiment'], prefix='sent')
df = pd.concat([df, sentiment_dummies], axis=1)

In [ ]:
df.head(2)

# Tokenize

In [ ]:
df['tokens'] = df['cleaned_text'].apply(toke)

# Lemmatize/Stop Word Removal

Three custom functions: `remove_stopwords`, `lemmatize_text`, `unlist` will be used to process the word tokens we created.

In [ ]:
pre_process = [remove_stopwords, lemmatize_text, unlist]

for action in pre_process:
    df.tokens = df.tokens.apply(action)
    print('Completed: {}'.format(str(action)))

In [ ]:
#Make sure there are no duplicate articles.
df.drop_duplicates(subset=['tokens'], inplace=True)

# Filtering out irellevant articles.

In [176]:
data = pd.read_pickle("/Users/brendanferris/Desktop/scripts/trading_on_sentiment/main_data/apple/seperated_data.pkl")

In [251]:
from collections import Counter

def relevant_news(article):
    t = re.findall(r'\b[A-Za-z]{2,25}\b', article.lower())
    relevant_keywords = dict(Counter(t))
    
    freq_dist = pd.DataFrame(list(relevant_keywords.items()), columns=['word', 'freq'])
    freq_dist = freq_dist.sort_values(by=['freq'], ascending=False)
    freq_dist = freq_dist.reset_index(drop=True)
    
    apple_mentions = freq_dist.loc[freq_dist['word'] == 'apple']
    stock_mentions = freq_dist.loc[(freq_dist['word'] == 'stock') | (freq_dist['word'] == 'stocks') | (freq_dist['word'] == 'market') | (freq_dist['word'] == 'wall') | (freq_dist['word'] == 'street') | (freq_dist['word'] == 'analyst')]
    company_mentions = freq_dist.loc[(freq_dist['word'] == 'company') | (freq_dist['word'] == 'tim')]
    product_mentions = freq_dist.loc[(freq_dist['word'] == 'iphone') | (freq_dist['word'] == 'phone') | (freq_dist['word'] == 'iphones') | (freq_dist['word'] == 'macbook') | (freq_dist['word'] == 'watch') | (freq_dist['word'] == 'ipad') | (freq_dist['word'] == 'itunes')]
    technology_mentions = freq_dist.loc[(freq_dist['word'] == 'technology') | (freq_dist['word'] == 'tech')]
    
    try:
        if int(sum(apple_mentions.freq.values)) >= 3 or int(sum(stock_mentions.freq.values)) >= 2 or int(sum(company_mentions.freq.values)) >= 1 or int(sum(product_mentions.freq.values)) > 1 or int(sum(technology_mentions.freq.values)) >= 1:
            return "POSSIBLE"
        else:
            return "PROBABLY NOT"
    except IndexError:
        return "PROBABLY NOT"
    

In [252]:
data['is_relevant'] = data['fulltext'].apply(relevant_news)

In [253]:
data.loc[data.is_relevant == 'PROBABLY NOT']

,update,source,author,fulltext,summary,title,cleaned_text,cleaned_authors,news_outlet,day_change,...,low,close,volume,sentiment,sent_negative,sent_neutral,sent_positive,tokens,total_articles,is_relevant
2015-01-04,2015-01-05 00:00:00,http://gizmodo.com/4k-drone-footage-captures-a...,[],"After looking at rendered images for years, se...",NaN,4K Drone Footage Captures Apple's New Spaceshi...,after looking at rendered images for years see...,,gizmodo,-0.7750,...,26.83750,27.3325,53204600.0,positive,0,0,1,"looking, rendered, image, year, seeing, real, ...",1,PROBABLY NOT
2015-01-08,2015-01-09 10:44:15+00:00,http://www.mirror.co.uk/news/uk-news/apple-ipo...,"['Pete Bainbridge', 'Image', 'Men']",Our free email newsletter sends you the bigges...,NaN,Apple iPods being used as SPY CAMERAS by thiev...,our free email newsletter sends you the bigges...,pete bainbridge image men,mirror,0.8600,...,27.17500,27.9725,59364500.0,negative,1,0,0,"free, sends, biggest, headline, sport, showbiz...",1,PROBABLY NOT
2015-01-11,2015-01-12 00:00:00,http://www.huffingtonpost.com/2015/01/12/apple...,['Senior Culture Reporter'],"Actually, this character is inspired from the ...",NaN,Here's A Bottle Of Apple Juice That Looks Some...,actually this character is inspired from the p...,senior culture reporter,huffingtonpost,-0.0175,...,27.55250,28.0025,53699500.0,positive,0,0,1,"actually, character, inspired, pre, modern, ja...",1,PROBABLY NOT
2015-01-13,2015-01-13 23:48:06+00:00,http://www.news.com.au/world/breaking-news/app...,[],Apple camera patent makes GoPro nervous\n\nAPP...,NaN,Apple camera patent makes GoPro nervous,apple camera patent makes gopro nervous apple ...,,news,-0.5975,...,27.22750,27.5550,67091900.0,negative,1,0,0,"apple, camera, patent, make, gopro, nervous, a...",1,PROBABLY NOT
2015-01-17,2015-01-18 10:28:01+00:00,http://www.inquisitr.com/1765290/mother-tried-...,['Addam Corré'],"A 29-year-old mother of two, who reportedly tr...",NaN,Mother Tried To Kill Her Kids With Drugged App...,a 29 year old mother of two who reportedly tri...,addam corr,inquisitr,0.2025,...,26.30000,26.4975,78513300.0,negative,1,0,0,"29, year, old, mother, two, reportedly, tried,...",1,PROBABLY NOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-16,2020-08-16 00:00:00,https://thetakeout.com/how-magical-is-apple-ci...,[],"Doesn’t it look magical, though? Photo : Madel...",NaN,"How magical is apple cider vinegar, really?",doesn t it look magical though photo madeleine...,,thetakeout,1.1900,...,113.04750,114.9075,17809052.0,positive,0,0,1,"look, magical, though, photo, madeleine, stein...",1,PROBABLY NOT
2020-08-16,2020-08-18 07:18:57+10:00,https://news.yahoo.com/fortnite-maker-says-app...,[],The Week\n\nNo one expected Donald Trump and h...,NaN,'Fortnite' maker says Apple threatened to cut ...,the week no one expected donald trump and his ...,,yahoo,1.1900,...,113.04750,114.9075,17809052.0,positive,0,0,1,"week, one, expected, donald, trump, supporter,...",1,PROBABLY NOT
2020-08-17,2020-08-18 00:00:00,https://www.wbtv.com/2020/08/18/apple-picking-...,"['Steve Ohnesorge', 'Published At Pm']",Officials say there’s simply not enough of the...,NaN,"Apple picking underway, pickers spreading out ...",officials say there s simply not enough of the...,steve ohnesorge published at pm,wbtv,-1.5700,...,113.97500,114.5825,15808241.0,neutral,0,1,0,"official, say, simply, enough, vaccine, everyo...",1,PROBABLY NOT
2020-08-18,2020-08-19 10:56:43+00:00,https://www.independent.co.uk/arts-entertainme...,[],Fiona Apple has spoken candidly about her feel...,NaN,Fiona Apple says ICE agents ‘must have failed ...,fiona apple has spoken candidly about her feel...,,independent,1.5525,...,114.02500,115.5625,13908263.0,positive,0,0,1,"fiona, apple, spoken, candidly, feeling, towar...",1,PROBABLY NOT


Another Approach (More general)

# Aggregate the daily news articles

We will perform modeling on the aggregated article text per day. Our data is in a format that has each row as a new article, we want to aggregate all of the articles on a given day into a single row.

In [ ]:
agged = df.copy()
agged.reset_index(inplace=True)
agged['date'] = pd.to_datetime(agged['index'])
agged.drop('index', axis=1, inplace=True)

In [ ]:
#Add a column of 1's for when we aggregate all info into one column, we can add the 1's later to get the total articles per day. 
agged['total_articles'] = 1

In [ ]:
sentiment = agged.groupby('date')['sent_negative', 'sent_positive', 'total_articles'].agg(np.sum)
text = agged.groupby('date')['tokens'].agg(''.join)

In [ ]:
agged = pd.merge(text, sentiment, how='inner', left_index=True, right_index=True)
#Merge targets
agged = pd.merge(agged, targs, how='inner', left_index=True, right_index=True)

In [ ]:
agged.head(2)

# Save the cleaned dataframe 

This dataframe will be used in the modeling process.

In [ ]:
# agged.to_pickle('main_data/gs/maindf.pkl')

In [ ]:
# df.to_pickle('main_data/gs/seperated_data.pkl')